In [1]:
import pandas as pd
import os

In [2]:
# 设置文件路径，导入路径
folder_path = r"E:\Test\Inputs"

file_coalfpplant = os.path.join(folder_path, "coal-fired_power_plants_China_wihout_ot_2020_operating.xlsx")
df_coalfpplant = pd.read_excel(file_coalfpplant)

file_gas_plant = os.path.join(folder_path, "gas-fired_power_plants_China_2020_operating.xlsx")
df_gas_plant = pd.read_excel(file_gas_plant)

file_wipp_plant = os.path.join(folder_path, "waste_inceration_plants_2020_operating.xlsx")
df_wipp_plant = pd.read_excel(file_wipp_plant)

# file_wipp = os.path.join(folder_path,"WIPP.xlsx")
# df_wipp = pd.read_excel(file_wipp)

# file_thermal_gen = os.path.join(folder_path,"province_thermal_power_generation_2020.xlsx")
# df_thermal_gen = pd.read_excel(file_thermal_gen)

# file_load = os.path.join(folder_path, "load_profile_monthly_province.csv")
# df_load = pd.read_csv(file_load)

file_water_int = os.path.join(folder_path,"province_intensity.xlsx")
df_water_int = pd.read_excel(file_water_int)


## 计算电厂年发电量（根据省年发电量、厂装机容量）

In [6]:
## 计算每个煤电厂的年发电量(kWh)

## 每个煤电厂的年发电量 = 省煤电发电总量（1e8 kWh）* 煤电厂装机容量（MW）/ 省煤电厂发电总装机容量（10MW）

# 创建一个字典，将province作为键，capacity_coal和gen_coal作为值
provincecoal_dict = dict(zip(df_thermal_gen['province'], zip(df_thermal_gen['capacity_coal'], df_thermal_gen['gen_coal'])))

# 定义一个函数来计算Annual_Gen
def calculate_annual_gen(row):
    province = row['Province']
    capacity_coal, gen_coal = provincecoal_dict[province]
    return row['Capacity_MW'] / capacity_coal / 10 * gen_coal * 1e8

# 在df_coalfpplant中新增一列Annual_Gen_kWh
df_coalfpplant['Annual_Gen_kWh'] = df_coalfpplant.apply(calculate_annual_gen, axis=1)

df_coalfpplant.to_excel(os.path.join(folder_path, "coal-fired_power_plants_China_wihout_ot_2020_operating.xlsx"), index=False)

In [15]:
province_gas_dict = dict(zip(df_thermal_gen['province'], zip(df_thermal_gen['capacity_gas'], df_thermal_gen['gen_gas'])))

def calculate_annual_gen_gas(row):
    province = row['Province']
    capacity_gas, gen_gas = province_gas_dict[province]
    return row['Capacity_MW'] / capacity_gas / 10 * gen_gas * 1e8

df_gaspplant['Annual_Gen_kWh'] = df_gaspplant.apply(calculate_annual_gen_gas, axis=1)

df_gaspplant.to_excel(os.path.join(folder_path, "gas-fired_power_plants_China_2020_operating.xlsx"), index=False)

## 计算电厂月发电量、耗水量（根据负荷曲线、区域耗水强度）

In [7]:
## 时间精度年→月 计算每个煤电厂的月发电量及发电煤耗，格式同污水厂数据
month_dict = {
    'Jan': '01',
    'Feb': '02',
    'Mar': '03',
    'Apr': '04',
    'May': '05',
    'Jun': '06',
    'Jul': '07',
    'Aug': '08',
    'Sep': '09',
    'Oct': '10',
    'Nov': '11',
    'Dec': '12'
}

df_CFPP_columns = ["Province","Plantcode", "Plantname", "Statistical month", "Capacity_MW", "Cooling_tech", "Combustion_tech", "Lifetime", "Monthly_Gen_kWh"]
df_CFPP = pd.DataFrame(columns=df_CFPP_columns)

for index, row in df_coalfpplant.iterrows():
    province = row['Province']
    plantcode = row['Plant_Code']
    plantname = row['Plantname']
    capacity = row['Capacity_MW']
    cooling_tech = row['Cooling_tech']
    combustion_tech = row['Combustion_tech']
    lifetime = row['Lifetime']
    annual_gen = row['Annual_Gen_kWh']

    for month in month_dict.keys():
        date_str = "2020" + month_dict[month] #生成日期字符串
        monthly_gen = annual_gen *  df_load.loc[df_load["Province"] == province, month].values[0] #获取特定省份、月份的发电量比例，计算月发电量
        df_CFPP.loc[len(df_CFPP)] = [province, plantcode, plantname, date_str, capacity, cooling_tech, combustion_tech, lifetime, monthly_gen]


### 计算月耗水量
provincecoalwater_dict = dict(zip(df_water_int['Province'], df_water_int['water_con_m3/kWh']))
Monthly_Water = df_CFPP.apply(lambda row: row['Monthly_Gen_kWh'] * provincecoalwater_dict.get(row['Province'], 0), axis=1)
df_CFPP['Monthly_Water_m3'] = Monthly_Water

df_CFPP.to_excel(os.path.join(folder_path, "CFPP_WITHOUT_OT.xlsx"), index=False)

In [16]:
month_dict = {
    'Jan': '01',
    'Feb': '02',
    'Mar': '03',
    'Apr': '04',
    'May': '05',
    'Jun': '06',
    'Jul': '07',
    'Aug': '08',
    'Sep': '09',
    'Oct': '10',
    'Nov': '11',
    'Dec': '12'
}

df_GFPP_columns = ["Province","Plantcode", "Plantname", "Statistical month", "Capacity_MW", "Tech", "Fuel","Lifetime", "Monthly_Gen_kWh"]
df_GFPP = pd.DataFrame(columns=df_GFPP_columns)

for index, row in df_gaspplant.iterrows():
    province = row['Province']
    plantcode = row['Plant_Code']
    plantname = row['Plant name (local script)']
    capacity = row['Capacity_MW']
    tech = row['Technology']
    fuel = row['Fuel']
    lifetime = row['Lifetime']
    annual_gen = row['Annual_Gen_kWh']

    for month in month_dict.keys():
        date_str = "2020" + month_dict[month] #生成日期字符串
        monthly_gen = annual_gen *  df_load.loc[df_load["Province"] == province, month].values[0] #获取特定省份、月份的发电量比例，计算月发电量
        df_GFPP.loc[len(df_GFPP)] = [province, plantcode, plantname, date_str, capacity, tech, fuel, lifetime, monthly_gen]


### 计算月耗水量
province_water_dict = dict(zip(df_water_int['Province'], df_water_int['water_con_m3/kWh']))
Monthly_Water = df_GFPP.apply(lambda row: row['Monthly_Gen_kWh'] * province_water_dict.get(row['Province'], 0), axis=1)
df_GFPP['Monthly_Water_m3'] = Monthly_Water

df_GFPP.to_excel(os.path.join(folder_path, "GFPP.xlsx"), index=False)

In [3]:
### 计算WIPP电厂月耗水量
province_water_dict = dict(zip(df_water_int['Province'], df_water_int['water_con_m3/kWh']))
Monthly_Water = df_wipp.apply(lambda row: row['Monthly_Gen_kWh'] * province_water_dict.get(row['Province'], 0), axis=1)
df_wipp['Monthly_Water_m3'] = Monthly_Water

df_wipp.to_excel(os.path.join(folder_path, "WIPP.xlsx"), index=False)

In [6]:
### 计算电厂年耗水量 

# df_coalfpplant
# df_gas_plant
# df_wipp_plant
province_water_dict = dict(zip(df_water_int['Province'], df_water_int['water_con_m3/kWh']))
Yearly_Water = df_wipp_plant.apply(lambda row: row['Annual_Gen_kWh'] * province_water_dict.get(row['Pro_en'], 0), axis=1)
df_wipp_plant['Yearly_Water_m3'] = Yearly_Water

df_wipp_plant.to_excel(os.path.join(folder_path, "waste_inceration_plants_2020_operating.xlsx"), index=False)

# file_coalfpplant = os.path.join(folder_path, "coal-fired_power_plants_China_wihout_ot_2020_operating.xlsx")
# file_gas_plant = os.path.join(folder_path, "gas-fired_power_plants_China_2020_operating.xlsx")
# file_wipp_plant = os.path.join(folder_path, "waste_inceration_plants_2020_operating.xlsx")

In [8]:
max_flux_coal = df_coalfpplant['Yearly_Water_m3'].max() / 1e6
max_flux_gas = df_gas_plant['Yearly_Water_m3'].max() / 1e6
max_flux_wipp = df_wipp_plant['Yearly_Water_m3'].max() / 1e6

In [9]:
max_flux_coal

80.59032314410477

In [10]:
max_flux_gas

29.1478988

In [11]:
max_flux_wipp

6.232207792207792